<a href="https://colab.research.google.com/github/carolvieirav/operalabs/blob/main/freightech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install & Import

In [ ]:
!pip install selenium

In [ ]:
!apt install chromium-chromedriver

In [33]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select

In [4]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [6]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

## class bot

In [100]:
class Bot:
  
  def __init__(self,username,password):
    self.username = username
    self.password = password
    self.bot = webdriver.Chrome('chromedriver',options=options)

  def login(self):
    bot = self.bot
    bot.get("https://freightech.ambev.com.br/#/login")
    bot.implicitly_wait(10) # seconds
    email = bot.find_element_by_xpath("/html/body/app-root/ft-core-login/div/div/div/wac-input[1]/div/div/input")
    password = bot.find_element_by_xpath("/html/body/app-root/ft-core-login/div/div/div/wac-input[2]/div/div/input")
    email.clear()
    password.clear()
    email.send_keys(self.username)
    password.send_keys(self.password)
    bot.find_element_by_xpath("/html/body/app-root/ft-core-login/div/div/div/wac-button[1]/button").click()
    print("Logged in Successfuly")

In [ ]:
automate = Bot("anne.pereira@operalog.com.br", "55026170") 
automate.login()

In [122]:
vigencia_dist =  '/html/body/app-root/ft-home/div/main/div/ft-dashboard/div[1]/div[2]/div[1]/wac-box/div'
chamadas_aber_dist = '/html/body/app-root/ft-home/div/main/div/ft-dashboard/div[1]/div[2]/div[2]/wac-box/div'
chamadas_resol_dist = '/html/body/app-root/ft-home/div/main/div/ft-dashboard/div[1]/div[2]/div[3]/wac-box/div'
chamadas_pen_dist = '/html/body/app-root/ft-home/div/main/div/ft-dashboard/div[1]/div[3]/div[1]/wac-box/div'
pen_alcada_dist = '/html/body/app-root/ft-home/div/main/div/ft-dashboard/div[1]/div[3]/div[2]/wac-box/div'


def meus_chamados(vigencia, chamadas_pen, chamadas_resol, chamadas_aber, pen_alcada):

  ''' 
  Extract dashboard's data for pages "distribuição", "apoio" e "T1" and store 
  in a pandas dataframe.
  '''

  paths = [vigencia, chamadas_pen, chamadas_resol, chamadas_aber, pen_alcada]

  data = []
  for path in paths:
    elements = automate.bot.find_elements_by_xpath(path)
    
    for elem in elements:
      automate.bot.implicitly_wait(10) # seconds
      data.append(elem.text)
      
  data = [item.replace("\n", " ") for item in data]
  #data = [pd.Series(item) for item in data]

  #df = pd.DataFrame(data)
  #df = df.T

  return 

meus_chamados(vigencia_dist, chamadas_pen_dist, chamadas_resol_dist, chamadas_aber_dist, pen_alcada_dist)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead


,0,1,2,3,4
0,Vigência RESTAM 9 DIAS,Chamados pendentes ROTA 0 CHAMADOS AS 0 CHAMAD...,Chamados resolvidos 30 CHAMADOS ROTA 30 Chamad...,Chamados em aberto 0 CHAMADOS ROTA 0 Chamados ...,Pendentes por alçada GOD 0 CHAMADOS CAMPO REGI...


## categoria chamados

In [220]:
# set implicit wait time
driver.implicitly_wait(10) # seconds

driver.get("https://freightech.ambev.com.br/#/categoria-chamados")

In [221]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [222]:
def columns_category():
  columns = [item.get_text().strip("\n") for item in soup.find_all('div',{'class': "ag-header-container"})]
  columns[0].replace("\n", " ")
  columns= columns[0].split()
  columns.pop(-2)
  columns.pop(-3)
  print(columns)

columns_category()

['Nome', 'Data', 'Usuário', 'AÇÕES']


In [223]:
def rows_category():
  rows = [item.get_text() for item in soup.find_all('div',{'class': "ag-body-viewport ag-layout-normal ag-row-no-animation"})]
  rows= rows[0].replace("\n", "")
  print(rows)

rows_category()

1. Sem evidência2021-05-13T08:58:05.495Nathalia Fernanda Viana Rodrigues2. Evidência incorreta2021-05-13T08:58:18.443Nathalia Fernanda Viana Rodrigues3. Sem evidência de validação2021-05-13T08:58:36.966Nathalia Fernanda Viana Rodrigues4. Sem aprovação prévia do GOD ou Regional via FreighTech2021-05-13T08:59:09.611Nathalia Fernanda Viana Rodrigues5. Ajuste realizado de forma centralizada2021-05-13T08:59:28.682Nathalia Fernanda Viana Rodrigues6. Divergência entre valor solicitado x valor da evidência2021-05-13T08:59:55.044Nathalia Fernanda Viana Rodrigues7. Abertura deve ser feita pelo Sisnec2021-05-13T09:00:13.265Nathalia Fernanda Viana Rodrigues8. Campo incorreto2021-05-13T09:00:21.916Nathalia Fernanda Viana Rodrigues9. Outros2021-05-14T10:50:23.645Sergi Blabia Garriga


## Exportação

In [240]:
# set implicit wait time
driver.implicitly_wait(10) # seconds

driver.get("https://freightech.ambev.com.br/#/exportacao-dados")
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [242]:
modelo = [item.get_text() for item in soup.find_all('p',{'class': "card-fav card-fav-custom no-fav is-selected"})]
modelo

[]